
# Section 1 · ChatGPT Clone (Colab Edition)

This notebook gives you a minimal, working ChatGPT-like loop plus a set of **policy-focused prompts** for your Vibe‑Coding Challenge.  
Run cells **top to bottom**.

**In today's section, you'll**
1. Add your API key (private to this runtime).  
2. Send your **first request** to the model.  
3. Use a small chat loop with **streaming tokens**.  
4. Try policy-oriented templates: haiku/poem, summary bot, interactive issue chat, and a debate simulator.

> Privacy note: Your API key stays in this runtime. Avoid sharing the notebook after entering your key.



## 1) Imports (just run)
No extra installs needed in Colab.


In [ ]:

import os
import time
from typing import List, Dict

import openai
from openai import OpenAI



## 2) Add your API key and set a system prompt
- If you already set `OPENAI_API_KEY` in the environment, this will pick it up.
- Otherwise you'll be prompted *once* (input is hidden).  
- You can customize the system prompt at any time.


In [ ]:

# ===============================
# API Key and System Prompt
# ===============================

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")

if not OPENAI_API_KEY:
    try:
        import getpass
        OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key (input hidden): ").strip()
    except Exception:
        raise ValueError("Unable to capture API key input. Set os.environ['OPENAI_API_KEY'] manually.")

if not OPENAI_API_KEY:
    raise ValueError("Missing API key. Please provide a valid OpenAI API key.")

# Make the key available to the SDK
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

# You can tweak this later (see Section 6)
SYSTEM_PROMPT = "You are a helpful assistant specialized in generative AI and public policy."



## 3) Configuration & global state
- Pick a model (you can change later).  
- Turn token streaming on/off.  
- A simple `conversation_history` stores the last messages.


In [ ]:

# ===============================
# Configuration
# ===============================

MODEL = "gpt-5-mini"   # You can swap to "gpt-4o" or "gpt-4o-mini" if available
STREAM_TOKENS = True   # stream tokens to the output

# Instantiate a single client for the session
client = OpenAI(api_key=OPENAI_API_KEY)

# Global conversation history (list of dicts with 'role' and 'content')
conversation_history: List[Dict[str, str]] = []

def reset_history() -> None:
    """Clear the global conversation history."""
    conversation_history.clear()



## 4) Core chat function (with streaming + simple retries)
**What this cell does**
- Builds a message list from the system prompt + recent history + your new user input.  
- Sends a request to the Chat Completions API.  
- If streaming is on, it prints tokens as they arrive.  
- Appends both user/assistant turns to `conversation_history`.

If you get a rate limit, it retries briefly with exponential backoff.


In [ ]:

def make_query(
    user_input: str,
    system_prompt: str = SYSTEM_PROMPT,
    model: str = MODEL,
    stream: bool = STREAM_TOKENS,
    max_history: int = 10,
    retries: int = 2,
    backoff_seconds: float = 2.0,
) -> str:
    """Send a query with context and return the assistant's text."""
    global conversation_history

    # Build messages (system + recent context + current user)
    context = conversation_history[-max_history:]
    messages = [{"role": "system", "content": system_prompt}] + context

    if not context or context[-1].get("role") != "user" or context[-1].get("content") != user_input:
        messages.append({"role": "user", "content": user_input})

    attempt = 0
    while True:
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=messages,
                stream=stream,
            )

            full_text = ""

            if stream:
                print("\nAssistant:", end=" ", flush=True)
                for chunk in resp:
                    if chunk.choices:
                        delta = chunk.choices[0].delta
                        if delta and getattr(delta, "content", None):
                            token = delta.content
                            print(token, end="", flush=True)
                            full_text += token
                print()  # newline after stream
            else:
                full_text = resp.choices[0].message.content
                print("\nAssistant:", full_text)

            # Update history
            conversation_history.append({"role": "user", "content": user_input})
            conversation_history.append({"role": "assistant", "content": full_text})
            return full_text

        except openai.RateLimitError as e:
            if attempt >= retries:
                print(f"\nRate limit error after {attempt+1} attempt(s): {e}")
                return ""
            delay = backoff_seconds * (2 ** attempt)
            print(f"\nRate limit. Retrying in {delay:.1f}s...")
            time.sleep(delay)
            attempt += 1
        except (openai.APIError, openai.APIConnectionError, openai.BadRequestError) as e:
            print(f"\nAPI error: {e}")
            return ""
        except Exception as e:
            print(f"\nUnexpected error: {e}")
            return ""



## 5) One-shot helper
Use `chat_once("your prompt")` for quick tests.


In [ ]:

def chat_once(prompt: str, **kwargs) -> str:
    """Convenience wrapper to send a single prompt and return text."""
    return make_query(prompt, **kwargs)



## 6) Your first API request
Just run this cell. If streaming is on, tokens appear as they arrive.


In [ ]:

_ = chat_once("Write a haiku about bananas.")



## 7) (Optional) Tweak the system prompt
- Change the tone or role of the assistant.  
- We reset the history so the new system prompt takes effect cleanly.


In [ ]:

SYSTEM_PROMPT = "You are a concise assistant for generative AI and public policy. Keep answers tight and specific."
reset_history()

_ = chat_once("In one paragraph, contrast broad LLMs with narrow task-specific models for policy analysis.")



# Part 2 · Vibe‑Coding Challenge (Policy)

**Goal:** Use generative AI as your *primary coder/author* to rapidly prototype ideas around a policy topic.

**What to try**
- Start simple (haiku/poem) and then move to summaries, debates, or an interactive issue chat.
- Modify prompts. Combine outputs. Iterate quickly.
- There’s no single correct approach; we’ll share takeaways at the end.

**Timebox:** Work until ~5–10 minutes before the end; be ready to share.



## 8) Policy‑themed haiku or poem (easy)
Change the topic to something you care about.


In [ ]:

_ = chat_once("Write a haiku about the challenges of urban housing policy.")



## 9) Policy summary bot (medium)
Paste a short excerpt (or a brief description) below. If the text is long, summarize the relevant parts first.


In [ ]:

policy_text = """
[Paste or briefly describe a policy, report, or issue here. If long, summarize key excerpts.]
"""

prompt = f"""
You are a policy analysis bot.
1) Read the policy content below.
2) Provide a concise summary (bulleted).
3) Suggest a creative, actionable recommendation (2-3 bullets).
4) Flag key tradeoffs and uncertainties.

POLICY CONTENT:
{policy_text}
"""

_ = chat_once(prompt)



## 10) Interactive “issue of interest” chat (hard)
Enter any policy issue; re-run as many times as you want.


In [ ]:

issue = input("Enter a policy issue of interest: ").strip()
_ = chat_once(f"Discuss the policy implications of {issue} in a creative and engaging manner.")



## 11) Debate simulator (easy → hard)
Start small with a pro/con. Then try two distinct 'agents' and a neutral moderator.


In [ ]:

# Simple version
_ = chat_once("Provide one argument in favor of and one argument against universal basic income.")

# Two-agent extension (sketch)
agent_a = "You are a policy analyst focused on efficiency and growth."
agent_b = "You are a policy analyst focused on equity and distribution."

reset_history()
_ = chat_once(f"As {agent_a} Give a short opening statement on universal basic income.")
_ = chat_once(f"As {agent_b} Respond with a short opening statement on universal basic income.")
_ = chat_once("As a neutral moderator, summarize the strongest point from each side and note the main point of disagreement.")



## 12) Optional: Minimal chat loop inside Colab
Type messages; type `exit` or `quit` (or interrupt the cell) to stop.


In [ ]:

reset_history()
print("Chat started. Type 'exit' to stop.\n")
while True:
    try:
        user_in = input("You: ").strip()
    except EOFError:
        break
    if user_in.lower() in {"exit", "quit"}:
        print("Goodbye!")
        break
    _ = make_query(user_in)



## 13) Optional: Responses API (one-shot)
For this section, the Chat Completions examples above are sufficient.  
If you want to experiment, here's a quick one-shot.


In [ ]:

from openai import OpenAI as _OpenAI
_client = _OpenAI()

resp = _client.responses.create(
    model=MODEL,
    input="Write a 2-sentence policy memo about congestion pricing."
)
print(resp.output_text)
